<a href="https://colab.research.google.com/github/Sinchana1997/Transfer_Learning/blob/master/Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pathlib import Path
import numpy as np
import joblib
from keras.preprocessing import image
from keras.applications import xception

# Empty lists to hold the images and labels for each each image
x_train = []
y_train = []

In [ ]:
!git clone https://github.com/Sinchana1997/Transfer_Learning.git

fatal: destination path 'Transfer_Learning' already exists and is not an empty directory.


In [ ]:
!ls

bird_feature_classifier_model.h5  Transfer_Learning  y_train.dat
Machine_Learning		  Transfer-Learning
sample_data			  x_train.dat


In [ ]:
# Load the training data set by looping over every image file
for image_file in Path("Transfer_Learning/training_dataset").glob("**/*.png"):

    # Load the current image file
    image_data = image.load_img(image_file, target_size=(73, 73))
    # Convert the loaded image file to a numpy array
    image_array = image.img_to_array(image_data)
    # Add the current image to our list of training images
    x_train.append(image_array)

    # Add a label for this image. If it was a middle image, label it 0. If it was a bird, label it 1.
    if "middle" in image_file.stem:
        y_train.append(0)
    else:
        y_train.append(1)

In [ ]:
pip install numpy

In [ ]:
# Convert the list of separate images into a single 4D numpy array. This is what Keras expects.
x_train = np.array(x_train)

# Normalize image data to 0-to-1 range
x_train = xception.preprocess_input(x_train)

# Load the pre-trained neural network to use as a feature extractor
feature_extractor = xception.Xception(weights='imagenet', include_top=False, input_shape=(73, 73, 3))

# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_train)

# Save the array of extracted features to a file
joblib.dump(features_x, "x_train.dat")

# Save the matching array of expected values to a file
joblib.dump(y_train, "y_train.dat")


['y_train.dat']

In [ ]:
# Load data set of extracted features
x_train = joblib.load("x_train.dat")
y_train = joblib.load("y_train.dat")


In [ ]:
Y_train = np.array(y_train)
print(x_train.shape[1:])
print(Y_train.shape)

(3, 3, 2048)
(2000,)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [ ]:
# Create a model and add layers
model = Sequential()

# Add layers to our model
model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(x_train,Y_train,epochs=10,shuffle=True, verbose=2)

# Save the trained model to a file so we can use it to make predictions later
model.save("bird_feature_classifier_model.h5")

Epoch 1/10
63/63 - 0s - loss: 0.1606 - accuracy: 0.9260
Epoch 2/10
63/63 - 0s - loss: 0.1548 - accuracy: 0.9345
Epoch 3/10
63/63 - 0s - loss: 0.1544 - accuracy: 0.9375
Epoch 4/10
63/63 - 0s - loss: 0.1320 - accuracy: 0.9470
Epoch 5/10
63/63 - 0s - loss: 0.1408 - accuracy: 0.9405
Epoch 6/10
63/63 - 0s - loss: 0.1051 - accuracy: 0.9600
Epoch 7/10
63/63 - 0s - loss: 0.1247 - accuracy: 0.9485
Epoch 8/10
63/63 - 0s - loss: 0.1084 - accuracy: 0.9545
Epoch 9/10
63/63 - 0s - loss: 0.0980 - accuracy: 0.9560
Epoch 10/10
63/63 - 0s - loss: 0.1179 - accuracy: 0.9500


In [ ]:
from keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Empty lists to hold the images and labels for each each image
x_test = []
y_test = []

# Load the test data set by looping over every image file
for image_file in Path("Transfer-Learning/test_dataset").glob("**/*.png"):

    # Load the current image file
    image_data = image.load_img(image_file, target_size=(73, 73))

    # Convert the loaded image file to a numpy array
    image_array = image.img_to_array(image_data)

    # Add the current image to our list of test images
    x_test.append(image_array)

    # Add an expected label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "not_bird" in image_file.stem:
        y_test.append(0)
    else:
        y_test.append(1)

# Convert the list of test images to a numpy array
x_test = np.array(x_test)

# Normalize test data set to 0-to-1 range
x_test = xception.preprocess_input(x_test)

In [ ]:
# Load our trained classifier model
model = load_model("bird_feature_classifier_model.h5")

# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_test)

# Given the extracted features, make a final prediction using our own model
predictions = model.predict(features_x)

# If the model is more than 50% sure the object is a bird, call it a bird.
# Otherwise, call it "not a bird".
predictions = predictions > 0.5

# Calculate how many mis-classifications the model makes
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

# Calculate Precision and Recall for each class
report = classification_report(y_test, predictions)
print(report)

True Positives: 91
True Negatives: 77
False Positives: 23
False Negatives: 9
              precision    recall  f1-score   support

           0       0.90      0.77      0.83       100
           1       0.80      0.91      0.85       100

    accuracy                           0.84       200
   macro avg       0.85      0.84      0.84       200
weighted avg       0.85      0.84      0.84       200

